In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
from yolo.models import YOLOv1
from yolo.data.VOC_Dataset import VOC_Dataset
from torch.utils.data import DataLoader
from utils.img_process import show_bbox

# output: 7 * 7 * 30
# S * S * ((x, y, w, h, confidence) * B=2 + C=20)
_voc_root = "/home/ubuntu/VOC2007/VOCdevkit/VOC2007/"
voc_ds = VOC_Dataset(_voc_root)
loader = DataLoader(voc_ds, batch_size=4, pin_memory=True)


(500, 375)
(448, 448, 3) [(0.526, 0.5626666666666666, 0.648, 0.904), (0.33, 0.704, 0.506, 0.992), (0.01, 0.6506666666666666, 0.134, 0.9973333333333333), (0.482, 0.5173333333333333, 0.59, 0.7973333333333333), (0.554, 0.496, 0.624, 0.5866666666666667)] [[263, 211, 324, 339], [165, 264, 253, 372], [5, 244, 67, 374], [241, 194, 295, 299], [277, 186, 312, 220]]


In [ ]:
import torch

test = torch.randn(3, 448, 448).unsqueeze(0)
yolo = YOLOv1()
res = yolo(test)